<a href="https://colab.research.google.com/github/hario27/PositionChecker/blob/master/3%E5%88%86%E8%B6%B3RSI%E3%83%88%E3%83%AC%E3%83%BC%E3%83%89_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

モジュール読み込み

In [81]:
!curl -L http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O && tar xzvf ta-lib-0.4.0-src.tar.gz
!cd ta-lib && ./configure --prefix=/usr && make && make install && cd - && pip install ta-lib
!pip install tqdm
!pip install plotly --upgrade

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   380  100   380    0     0   1696      0 --:--:-- --:--:-- --:--:--  1696
100   361  100   361    0     0    880      0 --:--:-- --:--:-- --:--:--  2299
100 1299k  100 1299k    0     0  1617k      0 --:--:-- --:--:-- --:--:-- 1617k
ta-lib/
ta-lib/config.sub
ta-lib/aclocal.m4
ta-lib/CHANGELOG.TXT
ta-lib/include/
ta-lib/include/ta_abstract.h
ta-lib/include/ta_func.h
ta-lib/include/ta_common.h
ta-lib/include/ta_config.h.in
ta-lib/include/Makefile.am
ta-lib/include/ta_libc.h
ta-lib/include/ta_defs.h
ta-lib/missing
ta-lib/ta-lib.spec.in
ta-lib/config.guess
ta-lib/Makefile.in
ta-lib/ta-lib.dpkg.in
ta-lib/Makefile.am
ta-lib/autogen.sh
ta-lib/install-sh
ta-lib/configure
ta-lib/depcomp
ta-lib/HISTORY.TXT
ta-lib/configure.in
ta-lib/autom4te.cache/
ta-lib/autom4te.cache/output.0
ta-lib/autom4te.cache/requests
ta-lib/autom4te.cache/outpu

In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import talib as ta

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


データ読み込み

In [83]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/price_data/USDJPY_M1_200701020000_201912312356.csv", sep='\t')
df.head()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
0,2007.01.02,00:00:00,118.98,118.98,118.94,118.94,8,0,30
1,2007.01.02,00:01:00,118.97,118.97,118.94,118.96,4,0,30
2,2007.01.02,00:02:00,118.97,118.97,118.94,118.95,5,0,30
3,2007.01.02,00:03:00,118.97,118.97,118.94,118.94,4,0,30
4,2007.01.02,00:04:00,118.97,118.97,118.95,118.95,3,0,30


In [84]:
df.describe()

,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
count,4.725383e+06,4.725383e+06,4.725383e+06,4.725383e+06,4.725383e+06,4.725383e+06,4.725383e+06
mean,1.019507e+02,1.019599e+02,1.019413e+02,1.019506e+02,4.711169e+01,1.776567e+07,1.490621e+01
std,1.346174e+01,1.346246e+01,1.346105e+01,1.346170e+01,5.070529e+01,5.386321e+07,1.062501e+01
min,7.560700e+01,7.562000e+01,7.557000e+01,7.560800e+01,1.000000e+00,0.000000e+00,1.000000e+00
25%,9.121200e+01,9.122200e+01,9.120000e+01,9.121100e+01,1.400000e+01,0.000000e+00,8.000000e+00
50%,1.047570e+02,1.047680e+02,1.047470e+02,1.047580e+02,3.200000e+01,0.000000e+00,1.100000e+01
75%,1.119570e+02,1.119640e+02,1.119490e+02,1.119570e+02,6.100000e+01,0.000000e+00,2.600000e+01
max,1.258130e+02,1.258530e+02,1.258000e+02,1.258120e+02,4.148000e+03,3.688250e+09,3.000000e+02


In [85]:
df.columns = ["date", "time", "open", "high", "low", "close", "tickvol", "vol", "spread"]
df = df.drop(["vol", "spread", "tickvol"], axis=1)
df.columns

Index(['date', 'time', 'open', 'high', 'low', 'close'], dtype='object')

In [86]:
# 日付のフォーマット変更及びindex化
df["datetime"] = df['date'] + " " + df["time"]
df["datetime"] = pd.to_datetime(df['datetime'])
df.set_index("datetime", inplace=True)
df.head()

,date,time,open,high,low,close
datetime,,,,,,
2007-01-02 00:00:00,2007.01.02,00:00:00,118.98,118.98,118.94,118.94
2007-01-02 00:01:00,2007.01.02,00:01:00,118.97,118.97,118.94,118.96
2007-01-02 00:02:00,2007.01.02,00:02:00,118.97,118.97,118.94,118.95
2007-01-02 00:03:00,2007.01.02,00:03:00,118.97,118.97,118.94,118.94
2007-01-02 00:04:00,2007.01.02,00:04:00,118.97,118.97,118.95,118.95


目的変数追加

In [87]:
df['diff'] = df["close"].shift(-15) - df['close']
df = df[:-15]
df.tail()

,date,time,open,high,low,close,diff
datetime,,,,,,,
2019-12-31 23:37:00,2019.12.31,23:37:00,108.634,108.638,108.633,108.636,-0.002
2019-12-31 23:38:00,2019.12.31,23:38:00,108.636,108.638,108.636,108.637,-0.004
2019-12-31 23:39:00,2019.12.31,23:39:00,108.637,108.638,108.637,108.638,-0.001
2019-12-31 23:40:00,2019.12.31,23:40:00,108.638,108.639,108.624,108.629,0.016
2019-12-31 23:41:00,2019.12.31,23:41:00,108.629,108.637,108.629,108.636,0.019


In [88]:
df["open"].min()

75.607

In [89]:
# 上昇したら「1」、下降したら「0」へデータを変換
df["target"] = 0
mask1 = df['diff'] >= 0.05
mask2 = df['diff'] <= -0.05
column_name = 'diff'
df.loc[mask1, "target"] = 1
df.loc[mask2, "target"] = -1
df.tail()

,date,time,open,high,low,close,diff,target
datetime,,,,,,,,
2019-12-31 23:37:00,2019.12.31,23:37:00,108.634,108.638,108.633,108.636,-0.002,0
2019-12-31 23:38:00,2019.12.31,23:38:00,108.636,108.638,108.636,108.637,-0.004,0
2019-12-31 23:39:00,2019.12.31,23:39:00,108.637,108.638,108.637,108.638,-0.001,0
2019-12-31 23:40:00,2019.12.31,23:40:00,108.638,108.639,108.624,108.629,0.016,0
2019-12-31 23:41:00,2019.12.31,23:41:00,108.629,108.637,108.629,108.636,0.019,0


In [90]:
df.resample("M").mean()

,open,high,low,close,diff,target
datetime,,,,,,
2007-01-31,120.371160,120.379590,120.361397,120.370211,0.001018,0.006856
2007-02-28,120.464794,120.472976,120.455385,120.463723,-0.001374,-0.009991
2007-03-31,117.274341,117.285230,117.262830,117.273769,-0.000588,0.006289
2007-04-30,118.862697,118.870762,118.854017,118.862208,0.001148,0.007372
2007-05-31,120.709602,120.717034,120.701649,120.709165,0.001308,0.008009
...,...,...,...,...,...,...
2019-08-31,106.248834,106.257844,106.239878,106.248791,-0.001276,-0.013367
2019-09-30,107.446511,107.452828,107.440327,107.446564,0.001010,0.009895
2019-10-31,108.137950,108.143629,108.132267,108.137958,-0.000036,-0.000725


テクニカル分析関数

説明変数追加

In [91]:
# 時間帯の傾向
df["hour"] = df.index.hour
df["dayweek"] = df.index.dayofweek
df.tail()

,date,time,open,high,low,close,diff,target,hour,dayweek
datetime,,,,,,,,,,
2019-12-31 23:37:00,2019.12.31,23:37:00,108.634,108.638,108.633,108.636,-0.002,0,23,1
2019-12-31 23:38:00,2019.12.31,23:38:00,108.636,108.638,108.636,108.637,-0.004,0,23,1
2019-12-31 23:39:00,2019.12.31,23:39:00,108.637,108.638,108.637,108.638,-0.001,0,23,1
2019-12-31 23:40:00,2019.12.31,23:40:00,108.638,108.639,108.624,108.629,0.016,0,23,1
2019-12-31 23:41:00,2019.12.31,23:41:00,108.629,108.637,108.629,108.636,0.019,0,23,1


In [92]:
df["takane"] = df["close"] - df["close"].rolling(window=15).max()
df["yasune"] = df["close"] - df["close"].rolling(window=15).min()

In [93]:
# 騰落率
df["rise_rate"] = (df["close"] - np.round(df["close"].rolling(window=15).mean(), 3)) / np.round(df["close"].rolling(window=15).mean(), 3) * 100
df.tail()

,date,time,open,high,low,close,diff,target,hour,dayweek,takane,yasune,rise_rate
datetime,,,,,,,,,,,,,
2019-12-31 23:37:00,2019.12.31,23:37:00,108.634,108.638,108.633,108.636,-0.002,0,23,1,-0.008,0.003,-0.001841
2019-12-31 23:38:00,2019.12.31,23:38:00,108.636,108.638,108.636,108.637,-0.004,0,23,1,-0.007,0.004,0.000000
2019-12-31 23:39:00,2019.12.31,23:39:00,108.637,108.638,108.637,108.638,-0.001,0,23,1,-0.006,0.005,0.000920
2019-12-31 23:40:00,2019.12.31,23:40:00,108.638,108.639,108.624,108.629,0.016,0,23,1,-0.015,0.000,-0.006444
2019-12-31 23:41:00,2019.12.31,23:41:00,108.629,108.637,108.629,108.636,0.019,0,23,1,-0.003,0.007,0.000921


In [94]:
period = 14
df['adx'] = ta.ADX(df["high"], df["low"], df["close"], timeperiod=14)
df['di+'] = ta.PLUS_DI(df["high"], df["low"], df["close"], timeperiod=14)
df['di-'] = ta.MINUS_DI(df["high"], df["low"], df["close"], timeperiod=14)
df.tail()

,date,time,open,high,low,close,diff,target,hour,dayweek,takane,yasune,rise_rate,adx,di+,di-
datetime,,,,,,,,,,,,,,,,
2019-12-31 23:37:00,2019.12.31,23:37:00,108.634,108.638,108.633,108.636,-0.002,0,23,1,-0.008,0.003,-0.001841,21.223165,24.349181,25.343213
2019-12-31 23:38:00,2019.12.31,23:38:00,108.636,108.638,108.636,108.637,-0.004,0,23,1,-0.007,0.004,0.000000,19.850108,23.377094,24.331441
2019-12-31 23:39:00,2019.12.31,23:39:00,108.637,108.638,108.637,108.638,-0.001,0,23,1,-0.006,0.005,0.000920,18.575127,22.885135,23.819398
2019-12-31 23:40:00,2019.12.31,23:40:00,108.638,108.639,108.624,108.629,0.016,0,23,1,-0.015,0.000,-0.006444,20.098894,17.079103,39.763952
2019-12-31 23:41:00,2019.12.31,23:41:00,108.629,108.637,108.629,108.636,0.019,0,23,1,-0.003,0.007,0.000921,21.513820,14.906916,34.706618


In [95]:
# SAR
df["sar"] = df["close"] - ta.SAR(df["high"], df["low"], 0.02, 0.2)
df.tail()

,date,time,open,high,low,close,diff,target,hour,dayweek,takane,yasune,rise_rate,adx,di+,di-,sar
datetime,,,,,,,,,,,,,,,,,
2019-12-31 23:37:00,2019.12.31,23:37:00,108.634,108.638,108.633,108.636,-0.002,0,23,1,-0.008,0.003,-0.001841,21.223165,24.349181,25.343213,0.003
2019-12-31 23:38:00,2019.12.31,23:38:00,108.636,108.638,108.636,108.637,-0.004,0,23,1,-0.007,0.004,0.000000,19.850108,23.377094,24.331441,0.004
2019-12-31 23:39:00,2019.12.31,23:39:00,108.637,108.638,108.637,108.638,-0.001,0,23,1,-0.006,0.005,0.000920,18.575127,22.885135,23.819398,0.005
2019-12-31 23:40:00,2019.12.31,23:40:00,108.638,108.639,108.624,108.629,0.016,0,23,1,-0.015,0.000,-0.006444,20.098894,17.079103,39.763952,-0.010
2019-12-31 23:41:00,2019.12.31,23:41:00,108.629,108.637,108.629,108.636,0.019,0,23,1,-0.003,0.007,0.000921,21.513820,14.906916,34.706618,-0.003


In [96]:
upper, middle, lower = ta.BBANDS(df['close'], timeperiod=21)
df["bb_up"] = df["close"] - upper
df["bb_mid"] = df["close"] - middle
df["bb_lower"] = df["close"] - lower
df.tail()

,date,time,open,high,low,close,diff,target,hour,dayweek,takane,yasune,rise_rate,adx,di+,di-,sar,bb_up,bb_mid,bb_lower
datetime,,,,,,,,,,,,,,,,,,,,
2019-12-31 23:37:00,2019.12.31,23:37:00,108.634,108.638,108.633,108.636,-0.002,0,23,1,-0.008,0.003,-0.001841,21.223165,24.349181,25.343213,0.003,-0.013012,-0.003667,0.005679
2019-12-31 23:38:00,2019.12.31,23:38:00,108.636,108.638,108.636,108.637,-0.004,0,23,1,-0.007,0.004,0.000000,19.850108,23.377094,24.331441,0.004,-0.011535,-0.002333,0.006868
2019-12-31 23:39:00,2019.12.31,23:39:00,108.637,108.638,108.637,108.638,-0.001,0,23,1,-0.006,0.005,0.000920,18.575127,22.885135,23.819398,0.005,-0.010022,-0.001048,0.007926
2019-12-31 23:40:00,2019.12.31,23:40:00,108.638,108.639,108.624,108.629,0.016,0,23,1,-0.015,0.000,-0.006444,20.098894,17.079103,39.763952,-0.010,-0.018809,-0.009286,0.000238
2019-12-31 23:41:00,2019.12.31,23:41:00,108.629,108.637,108.629,108.636,0.019,0,23,1,-0.003,0.007,0.000921,21.513820,14.906916,34.706618,-0.003,-0.011118,-0.001905,0.007309


In [97]:
# MACDの計算
macd, macdsignal, macdhist = ta.MACD(df["close"], fastperiod=12, slowperiod=26, signalperiod=9)
df["macdhist"] = macdhist
df.tail()

,date,time,open,high,low,close,diff,target,hour,dayweek,takane,yasune,rise_rate,adx,di+,di-,sar,bb_up,bb_mid,bb_lower,macdhist
datetime,,,,,,,,,,,,,,,,,,,,,
2019-12-31 23:37:00,2019.12.31,23:37:00,108.634,108.638,108.633,108.636,-0.002,0,23,1,-0.008,0.003,-0.001841,21.223165,24.349181,25.343213,0.003,-0.013012,-0.003667,0.005679,-0.000039
2019-12-31 23:38:00,2019.12.31,23:38:00,108.636,108.638,108.636,108.637,-0.004,0,23,1,-0.007,0.004,0.000000,19.850108,23.377094,24.331441,0.004,-0.011535,-0.002333,0.006868,0.000266
2019-12-31 23:39:00,2019.12.31,23:39:00,108.637,108.638,108.637,108.638,-0.001,0,23,1,-0.006,0.005,0.000920,18.575127,22.885135,23.819398,0.005,-0.010022,-0.001048,0.007926,0.000551
2019-12-31 23:40:00,2019.12.31,23:40:00,108.638,108.639,108.624,108.629,0.016,0,23,1,-0.015,0.000,-0.006444,20.098894,17.079103,39.763952,-0.010,-0.018809,-0.009286,0.000238,0.000168
2019-12-31 23:41:00,2019.12.31,23:41:00,108.629,108.637,108.629,108.636,0.019,0,23,1,-0.003,0.007,0.000921,21.513820,14.906916,34.706618,-0.003,-0.011118,-0.001905,0.007309,0.000409


In [98]:
# RSIの計算
df["rsi14"] = ta.RSI(df['close'], timeperiod=14)
df.tail()

,date,time,open,high,low,close,diff,target,hour,dayweek,takane,yasune,rise_rate,adx,di+,di-,sar,bb_up,bb_mid,bb_lower,macdhist,rsi14
datetime,,,,,,,,,,,,,,,,,,,,,,
2019-12-31 23:37:00,2019.12.31,23:37:00,108.634,108.638,108.633,108.636,-0.002,0,23,1,-0.008,0.003,-0.001841,21.223165,24.349181,25.343213,0.003,-0.013012,-0.003667,0.005679,-0.000039,37.922594
2019-12-31 23:38:00,2019.12.31,23:38:00,108.636,108.638,108.636,108.637,-0.004,0,23,1,-0.007,0.004,0.000000,19.850108,23.377094,24.331441,0.004,-0.011535,-0.002333,0.006868,0.000266,40.311196
2019-12-31 23:39:00,2019.12.31,23:39:00,108.637,108.638,108.637,108.638,-0.001,0,23,1,-0.006,0.005,0.000920,18.575127,22.885135,23.819398,0.005,-0.010022,-0.001048,0.007926,0.000551,42.686146
2019-12-31 23:40:00,2019.12.31,23:40:00,108.638,108.639,108.624,108.629,0.016,0,23,1,-0.015,0.000,-0.006444,20.098894,17.079103,39.763952,-0.010,-0.018809,-0.009286,0.000238,0.000168,30.805954
2019-12-31 23:41:00,2019.12.31,23:41:00,108.629,108.637,108.629,108.636,0.019,0,23,1,-0.003,0.007,0.000921,21.513820,14.906916,34.706618,-0.003,-0.011118,-0.001905,0.007309,0.000409,43.886945


In [99]:
# ROCP
df["rocp10"] = ta.ROCP(df["close"], timeperiod=10)
df.tail()

,date,time,open,high,low,close,diff,target,hour,dayweek,takane,yasune,rise_rate,adx,di+,di-,sar,bb_up,bb_mid,bb_lower,macdhist,rsi14,rocp10
datetime,,,,,,,,,,,,,,,,,,,,,,,
2019-12-31 23:37:00,2019.12.31,23:37:00,108.634,108.638,108.633,108.636,-0.002,0,23,1,-0.008,0.003,-0.001841,21.223165,24.349181,25.343213,0.003,-0.013012,-0.003667,0.005679,-0.000039,37.922594,0.000009
2019-12-31 23:38:00,2019.12.31,23:38:00,108.636,108.638,108.636,108.637,-0.004,0,23,1,-0.007,0.004,0.000000,19.850108,23.377094,24.331441,0.004,-0.011535,-0.002333,0.006868,0.000266,40.311196,0.000028
2019-12-31 23:39:00,2019.12.31,23:39:00,108.637,108.638,108.637,108.638,-0.001,0,23,1,-0.006,0.005,0.000920,18.575127,22.885135,23.819398,0.005,-0.010022,-0.001048,0.007926,0.000551,42.686146,0.000000
2019-12-31 23:40:00,2019.12.31,23:40:00,108.638,108.639,108.624,108.629,0.016,0,23,1,-0.015,0.000,-0.006444,20.098894,17.079103,39.763952,-0.010,-0.018809,-0.009286,0.000238,0.000168,30.805954,-0.000083
2019-12-31 23:41:00,2019.12.31,23:41:00,108.629,108.637,108.629,108.636,0.019,0,23,1,-0.003,0.007,0.000921,21.513820,14.906916,34.706618,-0.003,-0.011118,-0.001905,0.007309,0.000409,43.886945,-0.000028


In [100]:
# 移動平均線
df["ema5"] = df["close"] - ta.EMA(df["close"], timeperiod=5)
df["ema25"] = df["close"] - ta.EMA(df["close"], timeperiod=25)
df["ema200"] = df["close"] - ta.EMA(df["close"], timeperiod=200)
df.tail()

,date,time,open,high,low,close,diff,target,hour,dayweek,takane,yasune,rise_rate,adx,di+,di-,sar,bb_up,bb_mid,bb_lower,macdhist,rsi14,rocp10,ema5,ema25,ema200
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-31 23:37:00,2019.12.31,23:37:00,108.634,108.638,108.633,108.636,-0.002,0,23,1,-0.008,0.003,-0.001841,21.223165,24.349181,25.343213,0.003,-0.013012,-0.003667,0.005679,-0.000039,37.922594,0.000009,0.001215,-0.005475,-0.002435
2019-12-31 23:38:00,2019.12.31,23:38:00,108.636,108.638,108.636,108.637,-0.004,0,23,1,-0.007,0.004,0.000000,19.850108,23.377094,24.331441,0.004,-0.011535,-0.002333,0.006868,0.000266,40.311196,0.000028,0.001477,-0.004131,-0.001421
2019-12-31 23:39:00,2019.12.31,23:39:00,108.637,108.638,108.637,108.638,-0.001,0,23,1,-0.006,0.005,0.000920,18.575127,22.885135,23.819398,0.005,-0.010022,-0.001048,0.007926,0.000551,42.686146,0.000000,0.001651,-0.002890,-0.000417
2019-12-31 23:40:00,2019.12.31,23:40:00,108.638,108.639,108.624,108.629,0.016,0,23,1,-0.015,0.000,-0.006444,20.098894,17.079103,39.763952,-0.010,-0.018809,-0.009286,0.000238,0.000168,30.805954,-0.000083,-0.004899,-0.010976,-0.009323
2019-12-31 23:41:00,2019.12.31,23:41:00,108.629,108.637,108.629,108.636,0.019,0,23,1,-0.003,0.007,0.000921,21.513820,14.906916,34.706618,-0.003,-0.011118,-0.001905,0.007309,0.000409,43.886945,-0.000028,0.001401,-0.003670,-0.002300


In [101]:
# 移動平均線乖離率
period = 15
#df["kairi"] =  ((df["close"] - df["close"].rolling(window=period).mean()) / df["close"].rolling(window=period).mean()) * 100
df.tail()

,date,time,open,high,low,close,diff,target,hour,dayweek,takane,yasune,rise_rate,adx,di+,di-,sar,bb_up,bb_mid,bb_lower,macdhist,rsi14,rocp10,ema5,ema25,ema200
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-31 23:37:00,2019.12.31,23:37:00,108.634,108.638,108.633,108.636,-0.002,0,23,1,-0.008,0.003,-0.001841,21.223165,24.349181,25.343213,0.003,-0.013012,-0.003667,0.005679,-0.000039,37.922594,0.000009,0.001215,-0.005475,-0.002435
2019-12-31 23:38:00,2019.12.31,23:38:00,108.636,108.638,108.636,108.637,-0.004,0,23,1,-0.007,0.004,0.000000,19.850108,23.377094,24.331441,0.004,-0.011535,-0.002333,0.006868,0.000266,40.311196,0.000028,0.001477,-0.004131,-0.001421
2019-12-31 23:39:00,2019.12.31,23:39:00,108.637,108.638,108.637,108.638,-0.001,0,23,1,-0.006,0.005,0.000920,18.575127,22.885135,23.819398,0.005,-0.010022,-0.001048,0.007926,0.000551,42.686146,0.000000,0.001651,-0.002890,-0.000417
2019-12-31 23:40:00,2019.12.31,23:40:00,108.638,108.639,108.624,108.629,0.016,0,23,1,-0.015,0.000,-0.006444,20.098894,17.079103,39.763952,-0.010,-0.018809,-0.009286,0.000238,0.000168,30.805954,-0.000083,-0.004899,-0.010976,-0.009323
2019-12-31 23:41:00,2019.12.31,23:41:00,108.629,108.637,108.629,108.636,0.019,0,23,1,-0.003,0.007,0.000921,21.513820,14.906916,34.706618,-0.003,-0.011118,-0.001905,0.007309,0.000409,43.886945,-0.000028,0.001401,-0.003670,-0.002300


In [102]:
# 標準偏差
#df['sd5'] = df["close"] - ta.STDDEV(df["close"], timeperiod=5)
#df['sd10'] = df["close"] - ta.STDDEV(df["close"], timeperiod=10)
#df['sd15'] = df["close"] - ta.STDDEV(df["close"], timeperiod=15)
df.tail()

,date,time,open,high,low,close,diff,target,hour,dayweek,takane,yasune,rise_rate,adx,di+,di-,sar,bb_up,bb_mid,bb_lower,macdhist,rsi14,rocp10,ema5,ema25,ema200
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-31 23:37:00,2019.12.31,23:37:00,108.634,108.638,108.633,108.636,-0.002,0,23,1,-0.008,0.003,-0.001841,21.223165,24.349181,25.343213,0.003,-0.013012,-0.003667,0.005679,-0.000039,37.922594,0.000009,0.001215,-0.005475,-0.002435
2019-12-31 23:38:00,2019.12.31,23:38:00,108.636,108.638,108.636,108.637,-0.004,0,23,1,-0.007,0.004,0.000000,19.850108,23.377094,24.331441,0.004,-0.011535,-0.002333,0.006868,0.000266,40.311196,0.000028,0.001477,-0.004131,-0.001421
2019-12-31 23:39:00,2019.12.31,23:39:00,108.637,108.638,108.637,108.638,-0.001,0,23,1,-0.006,0.005,0.000920,18.575127,22.885135,23.819398,0.005,-0.010022,-0.001048,0.007926,0.000551,42.686146,0.000000,0.001651,-0.002890,-0.000417
2019-12-31 23:40:00,2019.12.31,23:40:00,108.638,108.639,108.624,108.629,0.016,0,23,1,-0.015,0.000,-0.006444,20.098894,17.079103,39.763952,-0.010,-0.018809,-0.009286,0.000238,0.000168,30.805954,-0.000083,-0.004899,-0.010976,-0.009323
2019-12-31 23:41:00,2019.12.31,23:41:00,108.629,108.637,108.629,108.636,0.019,0,23,1,-0.003,0.007,0.000921,21.513820,14.906916,34.706618,-0.003,-0.011118,-0.001905,0.007309,0.000409,43.886945,-0.000028,0.001401,-0.003670,-0.002300


In [103]:
#df["tsf14"] = df["close"] - ta.TSF(df["close"], timeperiod=14)

In [104]:
#df["linearreg14"] = df["close"] - ta.LINEARREG(df["close"], timeperiod=14)

In [105]:
# 入れ替え及び削除
df = df.drop(["date", "time", "open", "high", "low", "close", "diff"], axis=1)
#df = df.loc[:,['target', "hour", "dayweek", "rise_rate", "adx", "di+", "di-", "sar", "bb_up", "bb_mid", "bb_lower", "macdhist", "rsi14", "rsi21", "rsi30", "sma5", "sma25", "sma200", "rocp10", "rocp15", "rocp20", "rocp50", "kairi", "sd", "sd50","tsf14","tsf21", "tsf20"]]
df.tail()

,target,hour,dayweek,takane,yasune,rise_rate,adx,di+,di-,sar,bb_up,bb_mid,bb_lower,macdhist,rsi14,rocp10,ema5,ema25,ema200
datetime,,,,,,,,,,,,,,,,,,,
2019-12-31 23:37:00,0,23,1,-0.008,0.003,-0.001841,21.223165,24.349181,25.343213,0.003,-0.013012,-0.003667,0.005679,-0.000039,37.922594,0.000009,0.001215,-0.005475,-0.002435
2019-12-31 23:38:00,0,23,1,-0.007,0.004,0.000000,19.850108,23.377094,24.331441,0.004,-0.011535,-0.002333,0.006868,0.000266,40.311196,0.000028,0.001477,-0.004131,-0.001421
2019-12-31 23:39:00,0,23,1,-0.006,0.005,0.000920,18.575127,22.885135,23.819398,0.005,-0.010022,-0.001048,0.007926,0.000551,42.686146,0.000000,0.001651,-0.002890,-0.000417
2019-12-31 23:40:00,0,23,1,-0.015,0.000,-0.006444,20.098894,17.079103,39.763952,-0.010,-0.018809,-0.009286,0.000238,0.000168,30.805954,-0.000083,-0.004899,-0.010976,-0.009323
2019-12-31 23:41:00,0,23,1,-0.003,0.007,0.000921,21.513820,14.906916,34.706618,-0.003,-0.011118,-0.001905,0.007309,0.000409,43.886945,-0.000028,0.001401,-0.003670,-0.002300


訓練データとテストデータの取得

In [106]:
from sklearn.model_selection import train_test_split

data = df.drop("target", axis=1)
target = df["target"]

X_train, X_test, y_train, y_test = train_test_split(data, target, shuffle=False, random_state=0, test_size=0.01)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(np.sum(y_train == 1))
print(np.sum(y_train == 0))
print(np.sum(y_train == -1))

(4678114, 18)
(47254, 18)
(4678114,)
(47254,)
650919
3383508
643687


In [107]:
!pip install optuna
"""
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
"""

'\nfrom sklearn.preprocessing import StandardScaler\nsc = StandardScaler()\nX_train = sc.fit_transform(X_train)\n'

xgboost学習

In [108]:
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb
from sklearn.metrics import accuracy_score
import pickle

In [115]:
# モデルの作成
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

#from sklearn.model_selection import KFold
#import optuna.integration.lightgbm as lgb

# ハイパーパラメータサーチ&モデル構築
params = {} 
params ['learning_rate'] = 0.01
params ['boosting_type'] = 'gbdt'
params ['objective'] = 'multiclass'
params ['metric' ] = 'multi_logloss'
params ['max_depth'] = 5
params ['num_class'] = 3
#params ["n_estimators"] = 1000

#lgb_clf = lgb.LGBMClassifier(**params)
#lgb_clf.fit(X_train, y_train)
#model = lgb.train(params,trains,valid_sets=valids)

# モデルの学習
model = lgb.train(params,
train_set=lgb_train, # トレーニングデータの指定
valid_sets=lgb_eval # 検証データの指定
)

"""

"""
"""
# clf = lgb.LGBMClassifier()

# ハイパーパラメータ探索
# 'learning_rate': [0.11],
#  'max_depth': [5],
param = {
         'learning_rate': [0.11],
        'max_depth': [15],
         'n_estimators':[2000]
        }
# max_depth=12,
# n_estimators=2000
"""

"""
xgb_clf = xgb.XGBClassifier(
    tree_method='gpu_hist',
    max_depth=10,
    n_estimators=1000
)
xgb1 = xgb.XGBClassifier()
xgb_grid = GridSearchCV(xgb1, param, cv = 3)
xgb_grid.fit(X_train, y_train)
"""

LightGBMError: ignored

In [64]:
# 学習モデルの保存、読み込み
#pickle.dump(lgb_clf, open("/content/drive/My Drive/Colab Notebooks/" + "model.pkl", "wb"))

In [116]:
clf = pickle.load(open("/content/drive/My Drive/Colab Notebooks/" + "model.pkl", "rb"))
print(clf)
#print(clf.best_score)
#print(clf.feature_importance)
#print(clf.params)
#lgb.plot_importance(clf)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.01, max_depth=5,
               metric='multi_logloss', min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=1000,
               n_jobs=-1, num_class=3, num_leaves=31, objective='multiclass',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


In [ ]:
xgb_pred_train = clf.predict(X_train)
xgb_pred_test = clf.predict(X_test)
print("訓練データ")
print(accuracy_score(y_train ,xgb_pred_train))
print("テストデータ")
print(accuracy_score(y_test ,xgb_pred_test))

In [80]:
xgb_pred_train[:,0]

IndexError: ignored

In [67]:
# 成績の良いモデルを別で保存
# import pickle
# now = datetime.datetime.now()
# pickle.dump(xgb_grid.best_estimator_, open("/content/drive/My Drive/Colab Notebooks/" + "best_model_" + now.strftime('%Y%m%d_%H%M%S') + "now.pkl", "wb"))

In [68]:
"""
predict = clf.predict(X_train)
predict_valid = clf.predict(X_test)
y_pred = []
for x in predict:
    y_pred.append(np.argmax(x))

y_pred_valid = []
for x in predict_valid:
    y_pred_valid.append(np.argmax(x))

print("訓練データ")
print(accuracy_score(y_train ,y_pred))
print("テストデータ")
print(accuracy_score(y_test ,y_pred_valid))
"""
"""
xgb_pred_train = clf.predict(X_train)
xgb_pred_test = clf.predict(X_test)
print("訓練データ")
print(accuracy_score(y_train ,xgb_pred_train))
print("テストデータ")
print(accuracy_score(y_test ,xgb_pred_test))
"""

'\nxgb_pred_train = clf.predict(X_train)\nxgb_pred_test = clf.predict(X_test)\nprint("訓練データ")\nprint(accuracy_score(y_train ,xgb_pred_train))\nprint("テストデータ")\nprint(accuracy_score(y_test ,xgb_pred_test))\n'